In [7]:
import pandas as pd

# Define the condition for filtering (stations you're interested in)
column_name = 'station_number'  # Replace with your column name
value_conditions = [66037]  # List of station numbers to include (Sydney Airport)

# Define the columns to read (make sure to include 'station_number' in col_to_include)
col_to_include = ['station_number', 'datetime', 'air_temperature_in_degrees_c', 'wind_speed_in_km_h', 'relative_humidity_in_percentage']

# Specify the chunk size (e.g., 10000 rows per chunk)
chunk_size = 10000

# Initialize an empty list to store filtered chunks
filtered_chunks = []

# Iterate over the file in chunks
for chunk in pd.read_csv(r'c:\Users\z5404477\OneDrive - UNSW\04_Workspace\2. WIP\data\1. raw\BOM Weather Data\bom_data_nsw55167608.csv', 
                         usecols=col_to_include, chunksize=chunk_size):
    # Filter rows where 'station_number' is in the list of value_conditions
    filtered_chunk = chunk[chunk[column_name].isin(value_conditions)]
    filtered_chunks.append(filtered_chunk)

# Concatenate the filtered chunks into a single DataFrame
filtered_df = pd.concat(filtered_chunks)

# Display the first 5 filtered rows
print(filtered_df.head(5))


         station_number             datetime  air_temperature_in_degrees_c  \
1747282           66037  2022-12-01 00:00:00                          18.4   
1747283           66037  2022-12-01 00:30:00                          18.2   
1747284           66037  2022-12-01 01:00:00                          18.3   
1747285           66037  2022-12-01 01:30:00                          18.2   
1747286           66037  2022-12-01 02:00:00                          18.1   

         relative_humidity_in_percentage  wind_speed_in_km_h  
1747282                             63.0                13.0  
1747283                             65.0                13.0  
1747284                             66.0                13.0  
1747285                             68.0                13.0  
1747286                             69.0                13.0  


In [9]:
filtered_df['datetime'] = pd.to_datetime(filtered_df['datetime'], format='%Y-%m-%d %H:%M:%S')

In [10]:
filtered_df_2 = filtered_df[filtered_df['station_number'] == 66037] #use Sydney Airport station only

# make datetime the index
filtered_df_2.set_index('datetime', inplace=True)
#sort the index
filtered_df_2.sort_index(inplace=True)

In [11]:
filtered_df_2

,station_number,air_temperature_in_degrees_c,relative_humidity_in_percentage,wind_speed_in_km_h
datetime,,,,
2010-01-01 00:00:00,66037,23.6,84.0,14.8
2010-01-01 00:30:00,66037,22.9,89.0,14.8
2010-01-01 01:00:00,66037,23.4,86.0,14.8
2010-01-01 01:30:00,66037,23.3,87.0,14.8
2010-01-01 02:00:00,66037,23.6,84.0,9.4
...,...,...,...,...
2025-05-31 21:30:00,66037,14.7,86.0,9.4
2025-05-31 22:00:00,66037,14.7,84.0,7.6
2025-05-31 22:30:00,66037,14.7,84.0,5.4


In [12]:
#delete station_number column
filtered_df_2.drop(columns=['station_number'], inplace=True)

In [13]:
filtered_df_2

,air_temperature_in_degrees_c,relative_humidity_in_percentage,wind_speed_in_km_h
datetime,,,
2010-01-01 00:00:00,23.6,84.0,14.8
2010-01-01 00:30:00,22.9,89.0,14.8
2010-01-01 01:00:00,23.4,86.0,14.8
2010-01-01 01:30:00,23.3,87.0,14.8
2010-01-01 02:00:00,23.6,84.0,9.4
...,...,...,...
2025-05-31 21:30:00,14.7,86.0,9.4
2025-05-31 22:00:00,14.7,84.0,7.6
2025-05-31 22:30:00,14.7,84.0,5.4


In [14]:
#filter the datetime to include only from 1 July 2020 00:00:00 am to 30 June 2023 23:30:00, which is ASHD data
start_date = '2020-07-01 00:00:00'
end_date = '2023-06-30 23:30:00'
filtered_df = filtered_df[(filtered_df['datetime'] >= start_date) & (filtered_df['datetime'] <= end_date)]

# create a column of datetime from 1 July 2020 to 30 June 2023 with 30 minute intervals
expected_date_range = pd.date_range(start=start_date, end=end_date, freq='30min')

# make weather data df with the expected date range
weather_data_df = pd.DataFrame(index=expected_date_range)

# set  index name to 'datetime'
weather_data_df.index.name = 'datetime'

# merge the weather data with the filtered data, keeping both datetime from the weather data and the filtered data
weather_data_df = weather_data_df.merge(filtered_df_2, left_index=True, right_index=True, how='outer')

# fill NaN values the nearest previous value
weather_data_df.fillna(method='ffill', inplace=True)

# include only the row with expected date range
weather_data_df = weather_data_df.loc[expected_date_range]



#delete rows with duplicate datetime index
weather_data_df = weather_data_df[~weather_data_df.index.duplicated(keep='first')]

weather_data_df.to_csv(r'c:\Users\z5404477\OneDrive - UNSW\04_Workspace\2. WIP\data\2. processed\mascot_zs_weather_data.csv')

C:\Users\z5404477\AppData\Local\Temp\ipykernel_32276\3005662459.py:19: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  weather_data_df.fillna(method='ffill', inplace=True)


In [31]:
ds14 = pd.read_csv('../../../data/ds14_ausgrid_zs_mascot.csv', index_col=0, parse_dates=True)

In [32]:
ds15 = ds14[(ds14.index >= start_date) & (ds14.index <= end_date)]

In [33]:
ds15 = ds15.resample('30min').mean()

In [34]:
#merge the ds1 dataframe with the weather_data_df dataframe
ds15 = ds15.merge(weather_data_df, left_index=True, right_index=True, how='outer')

In [36]:
#check missing data in ds15
missing_data = ds15.isnull().sum()
print(missing_data[missing_data > 0])

Series([], dtype: int64)


In [37]:
ds15.to_csv('../../../data/ds15_ausgrid_zs_mascot_30min_with_weather.csv')